In [48]:
!pip install duckdb

In [49]:
import duckdb

In [50]:
con = duckdb.connect(database=':memory:')

In [51]:
# must be a table to use INSERT afterwards
con.execute("drop table if exists input").execute(
    "CREATE TABLE input AS "
    "SELECT id as rid, concat(given_name, ' ', surname, ' ', date_of_birth) as val "
    "FROM 'data/S1_clean_.csv'")

In [52]:
con.execute(
    "insert into input "
    "select id, concat(given_name, ' ', surname, ' ', date_of_birth) "
    "from 'data/S2_clean_.csv'")

In [53]:
con.execute(
    "insert into input "
    "select id, concat(given_name, ' ', surname, ' ', date_of_birth) "
    "from 'data/S3_clean_.csv'")

In [54]:
con.execute("select * from input").fetchall()

[('S1_0', 'joshua morrison 19101123'),
 ('S1_1', 'jordan white 19371126'),
 ('S1_2', 'emmerson lock 19211129'),
 ('S1_3', 'alexandra grosser 19720305'),
 ('S1_4', 'michael wuchatsch 19190110'),
 ('S1_5', 'emmerson loyck 19211129'),
 ('S1_6', 'rhys schuetz 19440909'),
 ('S1_7', 'joshua greenj 19790110'),
 ('S1_8', 'olivia hobson 19760812'),
 ('S1_9', 'michael lierach 19360816'),
 ('S1_10', 'elisabett domiten 19081008'),
 ('S1_11', 'genoveffa hylander 19071008'),
 ('S2_0', 'braecon schuetz 19440909'),
 ('S2_1', 'alexandra grosvenor 19930305'),
 ('S2_2', 'michael liersch 19360816'),
 ('S2_3', 'emmeron loyk 19321129'),
 ('S2_4', 'olivia hobson 19760812'),
 ('S2_5', 'joshua green 19010219'),
 ('S2_6', 'charlotte hyland 19340909'),
 ('S2_7', 'elisabet domitienn 19071008'),
 ('S3_0', 'emmerson loyck 19211129'),
 ('S3_1', 'michel wuchatsch 19190110'),
 ('S3_3', 'liersch michael 19360816'),
 ('S3_4', 'charlotte hyland 19460401'),
 ('S3_5', 'braedon schuetz 19440909'),
 ('S3_6', 'olivia hobson 1

In [55]:
q = 3

In [56]:
maxlen = con.execute("select max(length(val)) from input").fetchall()[0][0]

In [57]:
con.execute("drop view if exists tokens").execute(
    "create view tokens as "
    f"select rid, length(val) + {q} - 1 as rlen, "
    f"x as pos, substring(concat(repeat('#', {q} - 1), "
    "lower(val), "
    f"repeat('#',{q} - 1)),"
    f"x, {q}) as token "
    f"from input, range(1, {maxlen + q}) tbl(x) "
    f"where x <= length(val) + {q} - 1"
)

In [58]:
con.execute("select * from tokens").fetchall()

[('S2_1', 30, 30, '5##'),
 ('S2_1', 30, 29, '05#'),
 ('S1_11', 29, 29, '8##'),
 ('S2_7', 29, 29, '8##'),
 ('S2_1', 30, 28, '305'),
 ('S1_11', 29, 28, '08#'),
 ('S2_7', 29, 28, '08#'),
 ('S1_3', 28, 28, '5##'),
 ('S1_4', 28, 28, '0##'),
 ('S1_10', 28, 28, '8##'),
 ('S3_11', 28, 28, '8##'),
 ('S2_1', 30, 27, '030'),
 ('S1_11', 29, 27, '008'),
 ('S2_7', 29, 27, '008'),
 ('S1_3', 28, 27, '05#'),
 ('S1_4', 28, 27, '10#'),
 ('S1_10', 28, 27, '08#'),
 ('S3_11', 28, 27, '08#'),
 ('S2_6', 27, 27, '9##'),
 ('S3_1', 27, 27, '0##'),
 ('S3_4', 27, 27, '1##'),
 ('S2_1', 30, 26, '303'),
 ('S1_11', 29, 26, '100'),
 ('S2_7', 29, 26, '100'),
 ('S1_3', 28, 26, '305'),
 ('S1_4', 28, 26, '110'),
 ('S1_10', 28, 26, '008'),
 ('S3_11', 28, 26, '008'),
 ('S2_6', 27, 26, '09#'),
 ('S3_1', 27, 26, '10#'),
 ('S3_4', 27, 26, '01#'),
 ('S1_0', 26, 26, '3##'),
 ('S1_9', 26, 26, '6##'),
 ('S2_0', 26, 26, '9##'),
 ('S2_2', 26, 26, '6##'),
 ('S3_3', 26, 26, '6##'),
 ('S3_5', 26, 26, '9##'),
 ('S3_9', 26, 26, '3##'),
 (

In [59]:
t = 0.5

In [60]:
# brute force join, to get the gold standard
con.execute("drop view if exists bfjoin").execute(
    "create view bfjoin as "
    "select r1.rid as rid1, r2.rid as rid2 "
    # ", r1.rlen, r2.rlen, count(*) "
    # "from R as r1, R as r2 "
    "from tokens as r1, tokens as r2 "
    "where r1.token = r2.token "
    "and r1.rid < r2.rid "
    "group by r1.rid, r1.rlen, r2.rid, r2.rlen "
    f"having count(*) >= ceil({t} / (1+{t}) * (r1.rlen + r2.rlen))"
)

In [61]:
con.execute("select * from bfjoin").fetchall()

[('S1_11', 'S3_11'),
 ('S1_0', 'S3_9'),
 ('S1_9', 'S3_3'),
 ('S2_2', 'S3_3'),
 ('S1_5', 'S2_3'),
 ('S1_8', 'S3_6'),
 ('S2_4', 'S3_6'),
 ('S1_4', 'S3_1'),
 ('S1_7', 'S3_7'),
 ('S2_6', 'S3_4'),
 ('S1_10', 'S2_7'),
 ('S2_5', 'S3_7'),
 ('S1_5', 'S3_0'),
 ('S1_9', 'S2_2'),
 ('S2_0', 'S3_5'),
 ('S1_3', 'S2_1'),
 ('S1_8', 'S2_4'),
 ('S1_6', 'S3_5'),
 ('S1_2', 'S3_0'),
 ('S2_3', 'S3_0'),
 ('S1_6', 'S2_0'),
 ('S1_2', 'S1_5')]

## PrefixDF

In [62]:
con.execute("drop view if exists doc_frequency").execute(
    "CREATE VIEW doc_frequency AS "
    "SELECT token, count(*) AS df "
    "FROM tokens "
    "GROUP BY token "
    # "ORDER BY 2"
    # Save one ORDER BY clause here, it will be performed later
)

In [63]:
con.execute("SELECT * FROM doc_frequency").fetchall()

[('5##', 2),
 ('05#', 2),
 ('8##', 4),
 ('305', 2),
 ('08#', 4),
 ('0##', 5),
 ('030', 2),
 ('008', 4),
 ('10#', 5),
 ('9##', 9),
 ('1##', 1),
 ('303', 1),
 ('100', 4),
 ('110', 4),
 ('09#', 4),
 ('01#', 1),
 ('3##', 2),
 ('6##', 4),
 ('930', 1),
 ('710', 3),
 ('011', 6),
 ('909', 4),
 ('401', 1),
 ('23#', 2),
 ('16#', 3),
 ('993', 1),
 ('071', 3),
 ('203', 1),
 ('901', 5),
 ('810', 1),
 ('090', 4),
 ('040', 1),
 ('123', 2),
 ('816', 3),
 ('29#', 4),
 ('2##', 3),
 ('199', 1),
 ('907', 3),
 ('720', 1),
 ('190', 8),
 ('081', 7),
 ('409', 4),
 ('604', 1),
 ('112', 7),
 ('129', 4),
 ('12#', 3),
 ('972', 1),
 ('919', 2),
 ('908', 1),
 ('340', 1),
 ('460', 1),
 ('608', 6),
 ('812', 3),
 ('26#', 1),
 ('19#', 1),
 ('r 1', 3),
 ('197', 6),
 ('191', 4),
 ('934', 1),
 ('946', 1),
 ('101', 2),
 ('360', 3),
 ('440', 3),
 ('111', 3),
 ('126', 1),
 ('219', 1),
 ('410', 1),
 ('or ', 1),
 ('n 1', 9),
 ('193', 6),
 ('194', 4),
 ('910', 2),
 ('936', 3),
 ('944', 3),
 ('211', 4),
 ('021', 1),
 ('041', 1),

In [64]:
# Keep this select query to compare the output against the Table R, which will not keep 'df'
con.execute(
    "SELECT tt.*, df.df "
    "FROM tokens AS tt "
    "JOIN doc_frequency AS df ON tt.token = df.token "
    "ORDER BY tt.rid, df.df"
).fetchall()

[('S1_0', 26, 14, 'iso', 1),
 ('S1_0', 26, 13, 'ris', 1),
 ('S1_0', 26, 8, 'a m', 2),
 ('S1_0', 26, 10, 'mor', 2),
 ('S1_0', 26, 20, '910', 2),
 ('S1_0', 26, 9, ' mo', 2),
 ('S1_0', 26, 24, '123', 2),
 ('S1_0', 26, 11, 'orr', 2),
 ('S1_0', 26, 26, '3##', 2),
 ('S1_0', 26, 12, 'rri', 2),
 ('S1_0', 26, 21, '101', 2),
 ('S1_0', 26, 25, '23#', 2),
 ('S1_0', 26, 19, '191', 4),
 ('S1_0', 26, 6, 'hua', 5),
 ('S1_0', 26, 4, 'osh', 5),
 ('S1_0', 26, 3, 'jos', 5),
 ('S1_0', 26, 5, 'shu', 5),
 ('S1_0', 26, 7, 'ua ', 5),
 ('S1_0', 26, 2, '#jo', 6),
 ('S1_0', 26, 22, '011', 6),
 ('S1_0', 26, 1, '##j', 6),
 ('S1_0', 26, 23, '112', 7),
 ('S1_0', 26, 15, 'son', 7),
 ('S1_0', 26, 17, 'n 1', 9),
 ('S1_0', 26, 16, 'on ', 10),
 ('S1_0', 26, 18, ' 19', 30),
 ('S1_1', 23, 22, '26#', 1),
 ('S1_1', 23, 19, '711', 1),
 ('S1_1', 23, 5, 'rda', 1),
 ('S1_1', 23, 6, 'dan', 1),
 ('S1_1', 23, 3, 'jor', 1),
 ('S1_1', 23, 7, 'an ', 1),
 ('S1_1', 23, 17, '937', 1),
 ('S1_1', 23, 11, 'hit', 1),
 ('S1_1', 23, 21, '126', 

In [65]:
con.execute("drop view if exists R").execute(
    "CREATE VIEW R AS ("
    "SELECT tt.* "
    "FROM tokens AS tt "
    "JOIN doc_frequency AS df ON tt.token = df.token "
    "ORDER BY tt.rid, df.df"
    ")"
)

In [68]:
x = con.execute("select * from R").fetchall()
len(x), x

(771,
 [('S1_0', 26, 14, 'iso'),
  ('S1_0', 26, 13, 'ris'),
  ('S1_0', 26, 8, 'a m'),
  ('S1_0', 26, 10, 'mor'),
  ('S1_0', 26, 20, '910'),
  ('S1_0', 26, 9, ' mo'),
  ('S1_0', 26, 24, '123'),
  ('S1_0', 26, 11, 'orr'),
  ('S1_0', 26, 26, '3##'),
  ('S1_0', 26, 12, 'rri'),
  ('S1_0', 26, 21, '101'),
  ('S1_0', 26, 25, '23#'),
  ('S1_0', 26, 19, '191'),
  ('S1_0', 26, 6, 'hua'),
  ('S1_0', 26, 4, 'osh'),
  ('S1_0', 26, 3, 'jos'),
  ('S1_0', 26, 5, 'shu'),
  ('S1_0', 26, 7, 'ua '),
  ('S1_0', 26, 2, '#jo'),
  ('S1_0', 26, 22, '011'),
  ('S1_0', 26, 1, '##j'),
  ('S1_0', 26, 23, '112'),
  ('S1_0', 26, 15, 'son'),
  ('S1_0', 26, 17, 'n 1'),
  ('S1_0', 26, 16, 'on '),
  ('S1_0', 26, 18, ' 19'),
  ('S1_1', 23, 22, '26#'),
  ('S1_1', 23, 19, '711'),
  ('S1_1', 23, 5, 'rda'),
  ('S1_1', 23, 6, 'dan'),
  ('S1_1', 23, 3, 'jor'),
  ('S1_1', 23, 7, 'an '),
  ('S1_1', 23, 17, '937'),
  ('S1_1', 23, 11, 'hit'),
  ('S1_1', 23, 21, '126'),
  ('S1_1', 23, 10, 'whi'),
  ('S1_1', 23, 18, '371'),
  ('S1_1

In [36]:
# Keep this select query to compare the output against the Table PrefixR, which will not keep 'nrow'
x = con.execute(
    "select rid, rlen, pos, token, nrow "
    "from ("
    "SELECT R.*, row_number() OVER (PARTITION BY rid) as nrow "
    "FROM R "
    ")"
    f"where rlen - nrow + 1 >= ceil(rlen * {t}) "
).fetchall()
len(x), x

(409,
 [('S1_0', 26, 14, 'iso', 1),
  ('S1_0', 26, 1, '##j', 2),
  ('S1_0', 26, 8, 'a m', 3),
  ('S1_0', 26, 10, 'mor', 4),
  ('S1_0', 26, 20, '910', 5),
  ('S1_0', 26, 9, ' mo', 6),
  ('S1_0', 26, 24, '123', 7),
  ('S1_0', 26, 13, 'ris', 8),
  ('S1_0', 26, 26, '3##', 9),
  ('S1_0', 26, 12, 'rri', 10),
  ('S1_0', 26, 21, '101', 11),
  ('S1_0', 26, 25, '23#', 12),
  ('S1_0', 26, 19, '191', 13),
  ('S1_0', 26, 6, 'hua', 14),
  ('S1_1', 23, 8, 'n w', 1),
  ('S1_1', 23, 19, '711', 2),
  ('S1_1', 23, 5, 'rda', 3),
  ('S1_1', 23, 6, 'dan', 4),
  ('S1_1', 23, 3, 'jor', 5),
  ('S1_1', 23, 7, 'an ', 6),
  ('S1_1', 23, 17, '937', 7),
  ('S1_1', 23, 11, 'hit', 8),
  ('S1_1', 23, 21, '126', 9),
  ('S1_1', 23, 10, 'whi', 10),
  ('S1_1', 23, 18, '371', 11),
  ('S1_1', 23, 4, 'ord', 12),
  ('S1_10', 28, 10, 'tt ', 1),
  ('S1_10', 28, 20, ' 19', 2),
  ('S1_10', 28, 22, '908', 3),
  ('S1_10', 28, 17, 'ten', 4),
  ('S1_10', 28, 24, '810', 5),
  ('S1_10', 28, 14, 'omi', 6),
  ('S1_10', 28, 12, ' do', 7),

In [66]:
# no primary key on (id, token) yet
con.execute("drop view if exists PrefixDF").execute(
    "create view PrefixDF as "
    "select rid, rlen, pos, token "
    "from ("
    "SELECT R.*, row_number() OVER (PARTITION BY rid) as nrow "
    "FROM R "
    ")"
    # f"where nrow <= rlen - ceil(rlen * {t}) + 1 "
    f"where rlen - nrow + 1 >= ceil(rlen * {t}) "
)

In [67]:
x = con.execute("select * from PrefixDF").fetchall()
len(x), x

(409,
 [('S1_0', 26, 14, 'iso'),
  ('S1_0', 26, 1, '##j'),
  ('S1_0', 26, 8, 'a m'),
  ('S1_0', 26, 10, 'mor'),
  ('S1_0', 26, 20, '910'),
  ('S1_0', 26, 9, ' mo'),
  ('S1_0', 26, 24, '123'),
  ('S1_0', 26, 13, 'ris'),
  ('S1_0', 26, 26, '3##'),
  ('S1_0', 26, 12, 'rri'),
  ('S1_0', 26, 21, '101'),
  ('S1_0', 26, 25, '23#'),
  ('S1_0', 26, 19, '191'),
  ('S1_0', 26, 6, 'hua'),
  ('S1_1', 23, 8, 'n w'),
  ('S1_1', 23, 19, '711'),
  ('S1_1', 23, 5, 'rda'),
  ('S1_1', 23, 6, 'dan'),
  ('S1_1', 23, 3, 'jor'),
  ('S1_1', 23, 7, 'an '),
  ('S1_1', 23, 17, '937'),
  ('S1_1', 23, 11, 'hit'),
  ('S1_1', 23, 21, '126'),
  ('S1_1', 23, 10, 'whi'),
  ('S1_1', 23, 18, '371'),
  ('S1_1', 23, 4, 'ord'),
  ('S1_10', 28, 10, 'tt '),
  ('S1_10', 28, 20, ' 19'),
  ('S1_10', 28, 22, '908'),
  ('S1_10', 28, 17, 'ten'),
  ('S1_10', 28, 24, '810'),
  ('S1_10', 28, 14, 'omi'),
  ('S1_10', 28, 12, ' do'),
  ('S1_10', 28, 16, 'ite'),
  ('S1_10', 28, 15, 'mit'),
  ('S1_10', 28, 6, 'sab'),
  ('S1_10', 28, 11, 't 

## PrefixPos

In [70]:
con.execute("drop view if exists PrefixPos").execute(
    "create view PrefixPos as "
    "select rid, rlen, pos, token "
    "from tokens "
    # f"where nrow <= rlen - ceil(rlen * {t}) + 1 "
    f"where rlen - pos + 1 >= ceil(rlen * {t}) "
)

In [74]:
x = con.execute("select * from PrefixPos").fetchall()
len(x), x

(409,
 [('S2_1', 30, 16, 'sve'),
  ('S2_1', 30, 15, 'osv'),
  ('S1_11', 29, 15, 'lan'),
  ('S2_7', 29, 15, 'iti'),
  ('S1_3', 28, 15, 'oss'),
  ('S1_4', 28, 15, 'ats'),
  ('S1_10', 28, 15, 'mit'),
  ('S3_11', 28, 15, 'lan'),
  ('S2_1', 30, 14, 'ros'),
  ('S1_11', 29, 14, 'yla'),
  ('S2_7', 29, 14, 'mit'),
  ('S1_3', 28, 14, 'ros'),
  ('S1_4', 28, 14, 'hat'),
  ('S1_10', 28, 14, 'omi'),
  ('S3_11', 28, 14, 'lla'),
  ('S2_6', 27, 14, 'yla'),
  ('S3_1', 27, 14, 'ats'),
  ('S3_4', 27, 14, 'yla'),
  ('S1_0', 26, 14, 'iso'),
  ('S1_9', 26, 14, 'rac'),
  ('S2_0', 26, 14, 'uet'),
  ('S2_2', 26, 14, 'rsc'),
  ('S3_3', 26, 14, 'hae'),
  ('S3_5', 26, 14, 'uet'),
  ('S3_9', 26, 14, 'ios'),
  ('S2_1', 30, 13, 'gro'),
  ('S1_11', 29, 13, 'hyl'),
  ('S2_7', 29, 13, 'omi'),
  ('S1_3', 28, 13, 'gro'),
  ('S1_4', 28, 13, 'cha'),
  ('S1_10', 28, 13, 'dom'),
  ('S3_11', 28, 13, 'yll'),
  ('S2_6', 27, 13, 'hyl'),
  ('S3_1', 27, 13, 'hat'),
  ('S3_4', 27, 13, 'hyl'),
  ('S1_0', 26, 13, 'ris'),
  ('S1_9', 26

## Candidate set "DISTINCT"

In [99]:
con.execute("drop view if exists candset").execute(
    "CREATE VIEW candset AS ("
    "SELECT DISTINCT PR1.rid AS rid1, PR2.rid AS rid2 "
    "FROM PrefixPos PR1, PrefixPos PR2 "
    # "FROM PrefixDF PR1, PrefixDF PR2 "
    "WHERE PR1.token = PR2.token "
    "AND PR1.rid < PR2.rid "
    # length filter
    f"AND PR1.rlen >= ceil({t} * PR2.rlen)"
    f"AND PR1.rlen - PR1.pos + 1 >= CEIL(PR1.rlen * 2 * {t} / (1+{t})) " #?
    # positional filter
    "AND LEAST((PR1.rlen - PR1.pos + 1), (PR2.rlen - PR2.pos + 1)) >= "
    f"CEIL((PR1.rlen + PR2.rlen) * {t} / (1 + {t}))"
    ")"
)

In [100]:
con.execute("SELECT * FROM candset").fetchall()

[('S1_11', 'S3_11'),
 ('S1_4', 'S3_1'),
 ('S2_6', 'S3_4'),
 ('S1_0', 'S3_9'),
 ('S1_9', 'S2_2'),
 ('S2_0', 'S3_5'),
 ('S1_5', 'S2_3'),
 ('S1_2', 'S2_3'),
 ('S1_8', 'S3_6'),
 ('S2_4', 'S3_6'),
 ('S1_9', 'S3_1'),
 ('S2_2', 'S3_1'),
 ('S1_7', 'S3_7'),
 ('S2_5', 'S3_7'),
 ('S1_4', 'S2_2'),
 ('S1_0', 'S3_7'),
 ('S1_5', 'S3_3'),
 ('S3_0', 'S3_3'),
 ('S1_2', 'S3_3'),
 ('S1_4', 'S3_4'),
 ('S1_9', 'S3_4'),
 ('S2_2', 'S3_4'),
 ('S1_1', 'S3_7'),
 ('S1_10', 'S2_3'),
 ('S1_5', 'S2_0'),
 ('S1_4', 'S1_9'),
 ('S1_2', 'S1_5'),
 ('S1_0', 'S1_7'),
 ('S1_6', 'S3_5'),
 ('S1_5', 'S3_0'),
 ('S1_2', 'S3_0'),
 ('S2_3', 'S3_0'),
 ('S1_0', 'S1_1'),
 ('S2_7', 'S3_0'),
 ('S1_10', 'S3_0'),
 ('S1_7', 'S3_9'),
 ('S1_6', 'S2_0'),
 ('S2_5', 'S3_9'),
 ('S3_7', 'S3_9'),
 ('S1_1', 'S1_7'),
 ('S1_10', 'S1_5'),
 ('S2_0', 'S3_0'),
 ('S1_1', 'S3_9'),
 ('S1_10', 'S2_7'),
 ('S1_5', 'S2_7'),
 ('S1_2', 'S2_7'),
 ('S2_3', 'S2_7'),
 ('S1_3', 'S2_1'),
 ('S1_5', 'S3_5'),
 ('S3_0', 'S3_5'),
 ('S1_8', 'S2_4'),
 ('S1_7', 'S2_5'),
 ('S1_

In [101]:
con.execute("drop view if exists matches").execute(
    "create view matches as "
    "select r1.rid as rid1, r2.rid as rid2 "
    # ", count(*) "
    "from candset cs "
    "join R as r1 on r1.rid = cs.rid1 "
    "join R as r2 on r2.rid = cs.rid2 "
    "where r1.token = r2.token "
    "group by r1.rid, r1.rlen, r2.rid, r2.rlen "
    f"having count(*) >= ceil({t} / (1+{t}) * (r1.rlen + r2.rlen))"
)

In [102]:
con.execute("select *, row_number() over() from matches").fetchall()

[('S1_0', 'S3_9', 1),
 ('S1_10', 'S2_7', 2),
 ('S1_11', 'S3_11', 3),
 ('S1_2', 'S3_0', 4),
 ('S1_3', 'S2_1', 5),
 ('S1_4', 'S3_1', 6),
 ('S1_5', 'S3_0', 7),
 ('S1_6', 'S3_5', 8),
 ('S1_7', 'S3_7', 9),
 ('S1_8', 'S3_6', 10),
 ('S1_9', 'S2_2', 11),
 ('S2_0', 'S3_5', 12),
 ('S2_3', 'S3_0', 13),
 ('S2_4', 'S3_6', 14),
 ('S2_5', 'S3_7', 15),
 ('S2_6', 'S3_4', 16),
 ('S1_2', 'S1_5', 17),
 ('S1_5', 'S2_3', 18),
 ('S1_6', 'S2_0', 19),
 ('S1_8', 'S2_4', 20)]

In [103]:
con.execute(
    "select i1.val, i2.val "
    # ", i1.rid, i2.rid "
    ", row_number() over () "
    "from matches m "
    "join input i1 on i1.rid = m.rid1 "
    "join input i2 on i2.rid = m.rid2"
).fetchall()

[('joshua morrison 19101123', 'joshua morriosn 19101123', 1),
 ('genoveffa hylander 19071008', 'genovefa hyllande 19071008', 2),
 ('emmerson lock 19211129', 'emmerson loyck 19211129', 3),
 ('michael wuchatsch 19190110', 'michel wuchatsch 19190110', 4),
 ('emmerson loyck 19211129', 'emmerson loyck 19211129', 5),
 ('rhys schuetz 19440909', 'braedon schuetz 19440909', 6),
 ('joshua greenj 19790110', 'joshua green 19790110', 7),
 ('olivia hobson 19760812', 'olivia hobson 19760812', 8),
 ('michael lierach 19360816', 'michael liersch 19360816', 9),
 ('braecon schuetz 19440909', 'braedon schuetz 19440909', 10),
 ('emmeron loyk 19321129', 'emmerson loyck 19211129', 11),
 ('olivia hobson 19760812', 'olivia hobson 19760812', 12),
 ('joshua green 19010219', 'joshua green 19790110', 13),
 ('charlotte hyland 19340909', 'charlotte hyland 19460401', 14),
 ('emmerson lock 19211129', 'emmerson loyck 19211129', 15),
 ('emmerson loyck 19211129', 'emmeron loyk 19321129', 16),
 ('rhys schuetz 19440909', 'b

## Candidate set "GROUP BY"

In [24]:
# con.execute("drop view if exists candset2").execute(
#     "CREATE VIEW candset2 AS ("
#     "SELECT R1.rid AS rid1, R2.rid AS rid2 "
#     ", MAX(R1.pos) as maxPos1, MAX(R2.pos) as maxPos2, count(*) as prOverlap "
#     "FROM R R1, R R2 "
#     "WHERE R1.rid < R2.rid "
#     "AND R1.token = R2.token "
#     # length filter
#     f"AND R1.rlen >= ceil({t} * R2.rlen)"
#     # prefix filter
#     f"AND R1.rlen - R1.pos + 1 >= CEIL(R1.rlen * 2 * {t} / (1+{t})) " #?
#     f"AND R2.rlen - R2.pos + 1 >= CEIL(R2.rlen * {t}) "
#     # positional filter
#     "AND LEAST((R1.rlen - R1.pos + 1), (R2.rlen - R2.pos + 1)) >= "
#     f"CEIL((R1.rlen + R2.rlen) * {t} / (1 + {t})) "
#     "GROUP BY R1.rid, R2.rid "
#     ")"
# )

In [92]:
con.execute("drop view if exists candset2").execute(
    "CREATE VIEW candset2 AS ("
    "SELECT R1.rid AS rid1, R2.rid AS rid2 "
    ", MAX(R1.pos) as maxPos1, MAX(R2.pos) as maxPos2, count(*) as prOverlap "
    "FROM PrefixPos R1, PrefixPos R2 "
    "WHERE R1.rid < R2.rid "
    "AND R1.token = R2.token "
    # length filter
    f"AND R1.rlen >= ceil({t} * R2.rlen)"
    # prefix filter
    f"AND R1.rlen - R1.pos + 1 >= CEIL(R1.rlen * 2 * {t} / (1+{t})) " #?
    # positional filter
    "AND LEAST((R1.rlen - R1.pos + 1), (R2.rlen - R2.pos + 1)) >= "
    f"CEIL((R1.rlen + R2.rlen) * {t} / (1 + {t})) "
    "GROUP BY R1.rid, R2.rid "
    ")"
)

In [93]:
con.execute("select * from candset2").fetchall()

[('S1_11', 'S3_11', 10, 9, 8),
 ('S1_4', 'S3_1', 10, 9, 7),
 ('S2_6', 'S3_4', 10, 10, 10),
 ('S1_0', 'S3_9', 9, 9, 9),
 ('S1_9', 'S2_2', 9, 9, 9),
 ('S2_0', 'S3_5', 9, 9, 6),
 ('S1_5', 'S2_3', 9, 8, 6),
 ('S1_2', 'S2_3', 9, 8, 6),
 ('S1_8', 'S3_6', 9, 9, 9),
 ('S2_4', 'S3_6', 9, 9, 9),
 ('S1_9', 'S3_1', 8, 7, 5),
 ('S2_2', 'S3_1', 8, 7, 5),
 ('S1_7', 'S3_7', 8, 8, 8),
 ('S2_5', 'S3_7', 8, 8, 8),
 ('S1_4', 'S2_2', 8, 8, 8),
 ('S1_0', 'S3_7', 7, 7, 7),
 ('S1_5', 'S3_3', 6, 5, 1),
 ('S3_0', 'S3_3', 6, 5, 1),
 ('S1_2', 'S3_3', 6, 5, 1),
 ('S1_4', 'S3_4', 5, 3, 1),
 ('S1_9', 'S3_4', 5, 3, 1),
 ('S2_2', 'S3_4', 5, 3, 1),
 ('S1_1', 'S3_7', 2, 2, 2),
 ('S1_10', 'S2_3', 1, 1, 1),
 ('S1_5', 'S2_0', 9, 8, 1),
 ('S1_4', 'S1_9', 8, 8, 8),
 ('S1_2', 'S1_5', 8, 8, 8),
 ('S1_0', 'S1_7', 7, 7, 7),
 ('S1_6', 'S3_5', 7, 10, 1),
 ('S1_5', 'S3_0', 9, 9, 9),
 ('S1_2', 'S3_0', 8, 8, 8),
 ('S2_3', 'S3_0', 8, 9, 6),
 ('S1_0', 'S1_1', 2, 2, 2),
 ('S2_7', 'S3_0', 1, 1, 1),
 ('S1_10', 'S3_0', 1, 1, 1),
 ('S1_7', 

In [94]:
con.execute("drop view if exists matches2").execute(
    "create view matches2 as "
    "select r1.rid as rid1, r2.rid as rid2 "
    "from R r1, R r2, candset2 c "
    "where c.rid1 = r1.rid "
    "and c.rid2 = r2.rid "
    "and r1.token = r2.token "
    "and r1.pos > maxPos1 "
    "and r2.pos > maxPos2 "
    "group by r1.rid, r2.rid, r1.rlen, r2.rlen, prOverlap "
    f"having count(*) + prOverlap >= (r1.rlen + r2.rlen) * {t} / (1+{t})"
)

In [95]:
con.execute("select * from matches2").fetchall()

[('S1_0', 'S3_9'),
 ('S1_10', 'S2_7'),
 ('S1_11', 'S3_11'),
 ('S1_2', 'S3_0'),
 ('S1_3', 'S2_1'),
 ('S1_4', 'S3_1'),
 ('S1_5', 'S3_0'),
 ('S1_6', 'S3_5'),
 ('S1_7', 'S3_7'),
 ('S1_8', 'S3_6'),
 ('S1_9', 'S2_2'),
 ('S2_0', 'S3_5'),
 ('S2_3', 'S3_0'),
 ('S2_4', 'S3_6'),
 ('S2_5', 'S3_7'),
 ('S2_6', 'S3_4'),
 ('S1_2', 'S1_5'),
 ('S1_5', 'S2_3'),
 ('S1_6', 'S2_0'),
 ('S1_8', 'S2_4')]

# Debug

In [104]:
con.execute(
    "select * "
    "from matches m "
    "full outer join bfjoin b on b.rid1 = m.rid1 and b.rid2 = m.rid2 "
    "where m.rid1 is null "
).fetchall()

[(None, None, 'S1_9', 'S3_3'), (None, None, 'S2_2', 'S3_3')]

In [97]:
con.execute(
    "select * "
    "from matches2 m "
    "full outer join bfjoin b on b.rid1 = m.rid1 and b.rid2 = m.rid2 "
    "where m.rid1 is null "
).fetchall()

[(None, None, 'S1_9', 'S3_3'), (None, None, 'S2_2', 'S3_3')]

In [71]:
con.execute(
    "SELECT DISTINCT PR1.rid AS rid1, PR2.rid AS rid2 "
    "FROM PrefixR PR1, PrefixR PR2 "
    "WHERE PR1.token = PR2.token "
    "AND PR1.rid < PR2.rid "
    # length filter
    f"AND PR1.rlen >= ceil({t} * PR2.rlen) "
    # positional filter
    "AND LEAST((PR1.rlen - PR1.pos + 1), (PR2.rlen - PR2.pos + 1)) >= "
    f"CEIL((PR1.rlen + PR2.rlen) * {t} / (1 + {t}))"
    ""
).fetchall()

[('S1_0', 'S2_5'),
 ('S1_10', 'S2_7'),
 ('S1_2', 'S1_5'),
 ('S1_2', 'S2_3'),
 ('S1_3', 'S2_1'),
 ('S1_4', 'S2_2'),
 ('S1_5', 'S2_3'),
 ('S1_8', 'S2_4'),
 ('S1_9', 'S2_2')]

In [72]:
x = con.execute(
    "select * "
    "from PrefixR pr1 "
    "where pr1.rid = 'S1_6' "
).fetchall()
len(x), x

(12,
 [('S1_6', 23, 6, 's s'),
  ('S1_6', 23, 5, 'ys '),
  ('S1_6', 23, 1, '##r'),
  ('S1_6', 23, 4, 'hys'),
  ('S1_6', 23, 3, 'rhy'),
  ('S1_6', 23, 2, '#rh'),
  ('S1_6', 23, 14, 'z 1'),
  ('S1_6', 23, 9, 'chu'),
  ('S1_6', 23, 10, 'hue'),
  ('S1_6', 23, 17, '944'),
  ('S1_6', 23, 12, 'etz'),
  ('S1_6', 23, 13, 'tz ')])

In [74]:
x = con.execute(
    "select *, LEAST((R1.rlen - R1.pos + 1), (R2.rlen - R2.pos + 1)), "
    f"((R1.rlen + R2.rlen) * {t} / (1 + {t})) "
    "from R r1, R r2 "
    "where r1.token = r2.token "
    "and r1.rid = 'S1_6' "
    "and r2.rid = 'S2_0' "
).fetchall()
len(x), x

(17,
 [('S1_6', 23, 7, ' sc', 'S2_0', 26, 10, ' sc', 17, 16.333333333333332),
  ('S1_6', 23, 9, 'chu', 'S2_0', 26, 12, 'chu', 15, 16.333333333333332),
  ('S1_6', 23, 10, 'hue', 'S2_0', 26, 13, 'hue', 14, 16.333333333333332),
  ('S1_6', 23, 11, 'uet', 'S2_0', 26, 14, 'uet', 13, 16.333333333333332),
  ('S1_6', 23, 12, 'etz', 'S2_0', 26, 15, 'etz', 12, 16.333333333333332),
  ('S1_6', 23, 13, 'tz ', 'S2_0', 26, 16, 'tz ', 11, 16.333333333333332),
  ('S1_6', 23, 14, 'z 1', 'S2_0', 26, 17, 'z 1', 10, 16.333333333333332),
  ('S1_6', 23, 16, '194', 'S2_0', 26, 19, '194', 8, 16.333333333333332),
  ('S1_6', 23, 17, '944', 'S2_0', 26, 20, '944', 7, 16.333333333333332),
  ('S1_6', 23, 18, '440', 'S2_0', 26, 21, '440', 6, 16.333333333333332),
  ('S1_6', 23, 19, '409', 'S2_0', 26, 22, '409', 5, 16.333333333333332),
  ('S1_6', 23, 20, '090', 'S2_0', 26, 23, '090', 4, 16.333333333333332),
  ('S1_6', 23, 21, '909', 'S2_0', 26, 24, '909', 3, 16.333333333333332),
  ('S1_6', 23, 22, '09#', 'S2_0', 26, 25,

In [73]:
con.execute(
    "select *, LEAST((PR1.rlen - PR1.pos + 1), (PR2.rlen - PR2.pos + 1)), "
    f"CEIL((PR1.rlen + PR2.rlen) * {t} / (1 + {t})) "
    "from PrefixR pr1, PrefixR pr2 "
    "where pr1.token = pr2.token "
    "and pr1.rid = 'S1_6' "
    "and pr2.rid = 'S2_0' "
).fetchall()

[('S1_6', 23, 9, 'chu', 'S2_0', 26, 12, 'chu', 15, 17.0),
 ('S1_6', 23, 10, 'hue', 'S2_0', 26, 13, 'hue', 14, 17.0),
 ('S1_6', 23, 17, '944', 'S2_0', 26, 20, '944', 7, 17.0),
 ('S1_6', 23, 12, 'etz', 'S2_0', 26, 15, 'etz', 12, 17.0),
 ('S1_6', 23, 13, 'tz ', 'S2_0', 26, 16, 'tz ', 11, 17.0)]

In [113]:
# S1_9: michael,lierach,19360816
# S3_3: liersch,michael,19360816
x = con.execute(
    "select *, LEAST((R1.rlen - R1.pos + 1), (R2.rlen - R2.pos + 1)), "
    f"((R1.rlen + R2.rlen) * {t} / (1 + {t})) "
    "from R r1, R r2 "
    "where r1.token = r2.token "
    "and r1.rid = 'S1_9' "
    "and r2.rid = 'S3_3' "
    "order by r1.pos"
).fetchall()
len(x), x

(18,
 [('S1_9', 26, 3, 'mic', 'S3_3', 26, 11, 'mic', 16, 17.333333333333332),
  ('S1_9', 26, 4, 'ich', 'S3_3', 26, 12, 'ich', 15, 17.333333333333332),
  ('S1_9', 26, 5, 'cha', 'S3_3', 26, 13, 'cha', 14, 17.333333333333332),
  ('S1_9', 26, 6, 'hae', 'S3_3', 26, 14, 'hae', 13, 17.333333333333332),
  ('S1_9', 26, 7, 'ael', 'S3_3', 26, 15, 'ael', 12, 17.333333333333332),
  ('S1_9', 26, 8, 'el ', 'S3_3', 26, 16, 'el ', 11, 17.333333333333332),
  ('S1_9', 26, 11, 'lie', 'S3_3', 26, 3, 'lie', 16, 17.333333333333332),
  ('S1_9', 26, 12, 'ier', 'S3_3', 26, 4, 'ier', 15, 17.333333333333332),
  ('S1_9', 26, 16, 'ch ', 'S3_3', 26, 8, 'ch ', 11, 17.333333333333332),
  ('S1_9', 26, 18, ' 19', 'S3_3', 26, 18, ' 19', 9, 17.333333333333332),
  ('S1_9', 26, 19, '193', 'S3_3', 26, 19, '193', 8, 17.333333333333332),
  ('S1_9', 26, 20, '936', 'S3_3', 26, 20, '936', 7, 17.333333333333332),
  ('S1_9', 26, 21, '360', 'S3_3', 26, 21, '360', 6, 17.333333333333332),
  ('S1_9', 26, 22, '608', 'S3_3', 26, 22, '608

In [118]:
# S2_2: michael,liersch,19360816
# S3_3: liersch,michael,19360816
x = con.execute(
    "select *, LEAST((R1.rlen - R1.pos + 1), (R2.rlen - R2.pos + 1)), "
    f"((R1.rlen + R2.rlen) * {t} / (1 + {t})) "
    "from R r1, R r2 "
    "where r1.token = r2.token "
    "and r1.rid = 'S2_2' "
    "and r2.rid = 'S3_3' "
    "order by r1.pos"
).fetchall()
len(x), x

(21,
 [('S2_2', 26, 3, 'mic', 'S3_3', 26, 11, 'mic', 16, 17.333333333333332),
  ('S2_2', 26, 4, 'ich', 'S3_3', 26, 12, 'ich', 15, 17.333333333333332),
  ('S2_2', 26, 5, 'cha', 'S3_3', 26, 13, 'cha', 14, 17.333333333333332),
  ('S2_2', 26, 6, 'hae', 'S3_3', 26, 14, 'hae', 13, 17.333333333333332),
  ('S2_2', 26, 7, 'ael', 'S3_3', 26, 15, 'ael', 12, 17.333333333333332),
  ('S2_2', 26, 8, 'el ', 'S3_3', 26, 16, 'el ', 11, 17.333333333333332),
  ('S2_2', 26, 11, 'lie', 'S3_3', 26, 3, 'lie', 16, 17.333333333333332),
  ('S2_2', 26, 12, 'ier', 'S3_3', 26, 4, 'ier', 15, 17.333333333333332),
  ('S2_2', 26, 13, 'ers', 'S3_3', 26, 5, 'ers', 14, 17.333333333333332),
  ('S2_2', 26, 14, 'rsc', 'S3_3', 26, 6, 'rsc', 13, 17.333333333333332),
  ('S2_2', 26, 15, 'sch', 'S3_3', 26, 7, 'sch', 12, 17.333333333333332),
  ('S2_2', 26, 16, 'ch ', 'S3_3', 26, 8, 'ch ', 11, 17.333333333333332),
  ('S2_2', 26, 18, ' 19', 'S3_3', 26, 18, ' 19', 9, 17.333333333333332),
  ('S2_2', 26, 19, '193', 'S3_3', 26, 19, '193

In [119]:
x = con.execute(
    "select *, LEAST((R1.rlen - R1.pos + 1), (R2.rlen - R2.pos + 1)), "
    f"((R1.rlen + R2.rlen) * {t} / (1 + {t})) "
    "from R r1, R r2 "
    "where r1.token = r2.token "
    "and r1.rid = 'S1_5' "
    "and r2.rid = 'S2_3' "
    "order by r1.pos"
).fetchall()
len(x), x

(16,
 [('S1_5', 25, 1, '##e', 'S2_3', 23, 1, '##e', 23, 16.0),
  ('S1_5', 25, 2, '#em', 'S2_3', 23, 2, '#em', 22, 16.0),
  ('S1_5', 25, 3, 'emm', 'S2_3', 23, 3, 'emm', 21, 16.0),
  ('S1_5', 25, 4, 'mme', 'S2_3', 23, 4, 'mme', 20, 16.0),
  ('S1_5', 25, 5, 'mer', 'S2_3', 23, 5, 'mer', 19, 16.0),
  ('S1_5', 25, 9, 'on ', 'S2_3', 23, 8, 'on ', 16, 16.0),
  ('S1_5', 25, 10, 'n l', 'S2_3', 23, 9, 'n l', 15, 16.0),
  ('S1_5', 25, 11, ' lo', 'S2_3', 23, 10, ' lo', 14, 16.0),
  ('S1_5', 25, 12, 'loy', 'S2_3', 23, 11, 'loy', 13, 16.0),
  ('S1_5', 25, 16, 'k 1', 'S2_3', 23, 14, 'k 1', 10, 16.0),
  ('S1_5', 25, 17, ' 19', 'S2_3', 23, 15, ' 19', 9, 16.0),
  ('S1_5', 25, 20, '211', 'S2_3', 23, 19, '211', 5, 16.0),
  ('S1_5', 25, 22, '112', 'S2_3', 23, 20, '112', 4, 16.0),
  ('S1_5', 25, 23, '129', 'S2_3', 23, 21, '129', 3, 16.0),
  ('S1_5', 25, 24, '29#', 'S2_3', 23, 22, '29#', 2, 16.0),
  ('S1_5', 25, 25, '9##', 'S2_3', 23, 23, '9##', 1, 16.0)])

In [121]:
con.execute(
    "select *, LEAST((PR1.rlen - PR1.pos + 1), (PR2.rlen - PR2.pos + 1)), "
    f"CEIL((PR1.rlen + PR2.rlen) * {t} / (1 + {t})) "
    "from PrefixR pr1, PrefixR pr2 "
    "where pr1.token = pr2.token "
    "and pr1.rid = 'S1_5' "
    "and pr2.rid = 'S2_3' "
).fetchall()

[('S1_5', 25, 12, 'loy', 'S2_3', 23, 11, 'loy', 13, 16.0),
 ('S1_5', 25, 11, ' lo', 'S2_3', 23, 10, ' lo', 14, 16.0),
 ('S1_5', 25, 16, 'k 1', 'S2_3', 23, 14, 'k 1', 10, 16.0)]